In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import datetime
from pycontrails.datalib.ecmwf import ERA5
from pycontrails.models.emissions import Emissions
from pycontrails.core.met import MetDataArray, MetDataset
from pycontrails.core.met_var import (
    AirTemperature,
    SpecificHumidity,
)
from init_chem import ChemDataset
from boxm import BoxModel

In [2]:
# Initialise coord arrays
lon_bounds = (-180, 180) #np.arange(-180, 180, 5)
lat_bounds = (-90, 90) #np.arange(-90, 90, 5)

grid_met = 0.5
grid_chem = 5.0
met_pressure_levels = np.array([400, 300, 200, 100])

time = ("2022-01-01 00:00:00", "2022-01-01 08:00:00")
ts_met = "1H"
ts_disp = "5min"
ts_chem = "5min"

runtime = "24H"

In [3]:
# Import met data from ERA5
era5 = ERA5(
        time=time,
        timestep_freq=ts_met,
        variables=[
                "t",
                "q",
                "u",
                "v",
                "w",
                "z",
                "relative_humidity"
        ],
        pressure_levels=met_pressure_levels
)

In [4]:
# download data from ERA5 (or open from cache)
met = era5.open_metdataset()
met.data = met.data.transpose("latitude", "longitude", "level", "time", ...)

met

MetDataset with data:

<xarray.Dataset>
Dimensions:                              (latitude: 721, longitude: 1440,
                                          level: 4, time: 9)
Coordinates:
  * longitude                            (longitude) float64 -180.0 ... 179.8
  * latitude                             (latitude) float64 -90.0 ... 90.0
  * level                                (level) float64 100.0 200.0 300.0 400.0
  * time                                 (time) datetime64[ns] 2022-01-01 ......
    air_pressure                         (level) float32 1e+04 2e+04 3e+04 4e+04
    altitude                             (level) float32 1.618e+04 ... 7.185e+03
Data variables:
    air_temperature                      (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 4, 1), meta=np.ndarray>
    specific_humidity                    (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 4, 1), meta=np.ndarray>
    eastward_wind                        (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 4, 1), meta=np.ndarray>
    northward_wind                       (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 4, 1), meta=np.ndarray>
    lagrangian_tendency_of_air_pressure  (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 4, 1), meta=np.ndarray>
    geopotential                         (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 4, 1), meta=np.ndarray>
    relative_humidity                    (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 4, 1), meta=np.ndarray>
Attributes:
    Conventions:          CF-1.6
    history:              2023-09-20 12:15:22 GMT by grib_to_netcdf-2.25.1: /...
    pycontrails_version:  0.47.0
    met_source:           ERA5

In [5]:
type(met)

pycontrails.core.met.MetDataset

In [6]:
# initialise example chem MetDataset
chem = ChemDataset(
        lon_bounds=lon_bounds,
        lat_bounds=lat_bounds,
        met=met,
        time=time,
) #.init_chem()

type(chem)
#chem.data = chem.data.transpose("latitude", "longitude", "level", "time", ...)

init_chem.ChemDataset

In [7]:
chem.open_chemdataset()

/home/ktait98/pycontrails_kt/pycontrails/models/plume_chem/init_chem.py:142: UserWarning: Converting non-nanosecond precision timedelta values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  delta = offsets.astype('timedelta64[s]')


In [8]:
chem.data["Y"].isel(species=0, time=0, level=0, longitude=0).values

array([3.094e-13, 3.048e-13, 3.222e-13, 3.237e-13, 3.355e-13, 3.470e-13,
       3.598e-13, 3.386e-13, 3.832e-13])

In [9]:
chem.data["local_time"]

<xarray.DataArray 'local_time' (latitude: 36, longitude: 72, time: 9)>
array([[['2021-12-31T12:00:00.000000000',
         '2021-12-31T13:00:00.000000000',
         '2021-12-31T14:00:00.000000000', ...,
         '2021-12-31T18:00:00.000000000',
         '2021-12-31T19:00:00.000000000',
         '2021-12-31T20:00:00.000000000'],
        ['2021-12-31T12:20:00.000000000',
         '2021-12-31T13:20:00.000000000',
         '2021-12-31T14:20:00.000000000', ...,
         '2021-12-31T18:20:00.000000000',
         '2021-12-31T19:20:00.000000000',
         '2021-12-31T20:20:00.000000000'],
        ['2021-12-31T12:40:00.000000000',
         '2021-12-31T13:40:00.000000000',
         '2021-12-31T14:40:00.000000000', ...,
         '2021-12-31T18:40:00.000000000',
         '2021-12-31T19:40:00.000000000',
         '2021-12-31T20:40:00.000000000'],
        ...,
        ['2022-01-01T11:00:00.000000000',
...
         '2021-12-31T20:40:00.000000000'],
        ...,
        ['2022-01-01T11:00:00.000000000',
         '2022-01-01T12:00:00.000000000',
         '2022-01-01T13:00:00.000000000', ...,
         '2022-01-01T17:00:00.000000000',
         '2022-01-01T18:00:00.000000000',
         '2022-01-01T19:00:00.000000000'],
        ['2022-01-01T11:20:00.000000000',
         '2022-01-01T12:20:00.000000000',
         '2022-01-01T13:20:00.000000000', ...,
         '2022-01-01T17:20:00.000000000',
         '2022-01-01T18:20:00.000000000',
         '2022-01-01T19:20:00.000000000'],
        ['2022-01-01T11:40:00.000000000',
         '2022-01-01T12:40:00.000000000',
         '2022-01-01T13:40:00.000000000', ...,
         '2022-01-01T17:40:00.000000000',
         '2022-01-01T18:40:00.000000000',
         '2022-01-01T19:40:00.000000000']]], dtype='datetime64[ns]')
Coordinates:
  * latitude   (latitude) float64 -90.0 -85.0 -80.0 -75.0 ... 75.0 80.0 85.0
  * longitude  (longitude) float64 -180.0 -175.0 -170.0 ... 165.0 170.0 175.0
  * time       (time) datetime64[ns] 2022-01-01 ... 2022-01-01T08:00:00

In [10]:
for s in species:
    chem["Y"].data.loc[:, :, 201, time[0], s] = np.loadtxt("species/" + s + "_MONTH_1_LEVEL_8.csv",
                                                           delimiter=",")

NameError: name 'species' is not defined

In [ ]:
boxm = BoxModel(met, chem)

In [ ]:
boxm.met

In [ ]:
boxm.chem

In [ ]:
emi = xr.Dataset(
    {
        "EM": (["latitude", "longitude", "level", "time", "species"],
                np.zeros((len(latitude), len(longitude), len(level), len(time), len(species)))),
    },
    coords={
        "latitude": latitude,
        "longitude": longitude, 
        "level": level,
        "time": time,
        "species": species,
    }
)

In [ ]:
emi = MetDataset(emi)
emi = emi.data.transpose("latitude", "longitude", "level", "time", ...)

In [ ]:
emi

In [ ]:
boxm.params

In [ ]:
boxm.eval(source=emi)